In [59]:
# Dependencies
import requests
import json
from config import api_key , fromdate, todate
from opendata_config import par311, password, username
import pandas as pd
from sodapy import Socrata
from datetime import datetime

##### In the following script we'll try to analize the following

###### NYC311 can provide you access to non-emergency City services and information about City government programs. NYC311 can help with a broad range of services, including things like homeless person assistance, pothole repair, or help understanding a property tax bill. over 500 services

###### - which are the top 10 most popular report ploblems 311 receives (top 3 apartment mainrenance complaint, ilegal parking and noise from neighbor)
###### - what's the time gap they take to solve problems 
###### - what location report more issues in nyc 
###### - What burough has the highest level of 311 calls
###### - Were the 311 calls resolved (which burough has the highest resolution rate)
###### - Which category of service is the best/worst

In [63]:
# URL for GET requests to retrieve vehicle data
url_calendar= 'https://api.nyc.gov/public/api/GetCalendar?fromdate={fromdate}&todate={todate}'
headers = {'Ocp-Apim-Subscription-Key': api_key}

In [64]:
# Print the response object to the console
response = requests.get(url_calendar, headers=headers)
print(response.status_code)

500


In [36]:
# Retrieving data and converting it into JSON
response_json =response.json()

In [37]:
# Pretty Print the output of the JSON
print(json.dumps(response_json, indent=4, sort_keys=True))

{
    "Error": {
        "Code": "InternalServerError",
        "Message": "String '{fromdate}' was not recognized as a valid DateTime."
    }
}


In [28]:
# URL for GET requests to retrieve vehicle data
url_service = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json?$query=SELECT%0A%20%20%60unique_key%60%2C%0A%20%20%60created_date%60%2C%0A%20%20%60closed_date%60%2C%0A%20%20%60agency%60%2C%0A%20%20%60agency_name%60%2C%0A%20%20%60complaint_type%60%2C%0A%20%20%60descriptor%60%2C%0A%20%20%60location_type%60%2C%0A%20%20%60incident_zip%60%2C%0A%20%20%60incident_address%60%2C%0A%20%20%60street_name%60%2C%0A%20%20%60cross_street_1%60%2C%0A%20%20%60cross_street_2%60%2C%0A%20%20%60intersection_street_1%60%2C%0A%20%20%60intersection_street_2%60%2C%0A%20%20%60address_type%60%2C%0A%20%20%60city%60%2C%0A%20%20%60landmark%60%2C%0A%20%20%60facility_type%60%2C%0A%20%20%60status%60%2C%0A%20%20%60due_date%60%2C%0A%20%20%60resolution_description%60%2C%0A%20%20%60resolution_action_updated_date%60%2C%0A%20%20%60community_board%60%2C%0A%20%20%60bbl%60%2C%0A%20%20%60borough%60%2C%0A%20%20%60x_coordinate_state_plane%60%2C%0A%20%20%60y_coordinate_state_plane%60%2C%0A%20%20%60open_data_channel_type%60%2C%0A%20%20%60park_facility_name%60%2C%0A%20%20%60park_borough%60%2C%0A%20%20%60vehicle_type%60%2C%0A%20%20%60taxi_company_borough%60%2C%0A%20%20%60taxi_pick_up_location%60%2C%0A%20%20%60bridge_highway_name%60%2C%0A%20%20%60bridge_highway_direction%60%2C%0A%20%20%60road_ramp%60%2C%0A%20%20%60bridge_highway_segment%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%2C%0A%20%20%60%3A%40computed_region_efsh_h5xi%60%2C%0A%20%20%60%3A%40computed_region_f5dn_yrer%60%2C%0A%20%20%60%3A%40computed_region_yeji_bk3q%60%2C%0A%20%20%60%3A%40computed_region_92fq_4b7q%60%2C%0A%20%20%60%3A%40computed_region_sbqj_enih%60%2C%0A%20%20%60%3A%40computed_region_7mpf_4k6g%60%0AORDER%20BY%20%60created_date%60%20DESC%20NULL%20FIRST'
# headers = {'Ocp-Apim-Subscription-Key': api_key}

In [29]:
# Print the response object to the console
response_services = requests.get(url_service)
print(response.status_code)

500


In [84]:

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofnewyork.us", None)


# Define date range
start_date = datetime(2023, 1, 1).isoformat()  # January 1, 2023
end_date = datetime(2023, 1, 2).isoformat()  # June 30, 2024


# Example authenticated client (needed for non-public datasets):
client = Socrata(
    "data.cityofnewyork.us",
    par311,
    username= username,
    password= password,
    timeout=1000
)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# Query dataset with date filter
results = client.get("erm2-nwe9", where=f"created_date between '{start_date}' and '{end_date}'", 
                     limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [85]:
results_df.tail()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,latitude,longitude,location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,facility_type,taxi_pick_up_location,due_date
1995,56419469,2023-01-01T17:14:49.000,2023-01-31T07:58:57.000,EDC,Economic Development Corporation,Noise - Helicopter,Other,Above Address,11414,88-08 151 AVENUE,...,40.66848607627237,-73.84690627257683,"{'latitude': '40.66848607627237', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,56414163,2023-01-01T17:14:47.000,2023-02-10T07:50:33.000,HPD,Department of Housing Preservation and Develop...,WATER LEAK,HEAVY FLOW,RESIDENTIAL BUILDING,11211,293 HOOPER STREET,...,40.707299184747534,-73.95389367380741,"{'latitude': '40.707299184747534', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,56417321,2023-01-01T17:14:41.000,2023-01-01T18:09:05.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10002,70 GOUVERNEUR STREET,...,40.71254428253283,-73.98377512586437,"{'latitude': '40.71254428253283', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,56412966,2023-01-01T17:14:31.000,2023-01-02T03:12:30.000,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11385,56-23 COOPER AVENUE,...,40.693111410771586,-73.901388131749,"{'latitude': '40.693111410771586', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999,56412927,2023-01-01T17:14:29.000,2023-01-01T17:16:40.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,10472,1329 COMMONWEALTH AVENUE,...,40.83342849023004,-73.86865697421297,"{'latitude': '40.83342849023004', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 39 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   unique_key                      2000 non-null   object
 1   created_date                    2000 non-null   object
 2   closed_date                     1987 non-null   object
 3   agency                          2000 non-null   object
 4   agency_name                     2000 non-null   object
 5   complaint_type                  2000 non-null   object
 6   descriptor                      1984 non-null   object
 7   location_type                   1868 non-null   object
 8   incident_zip                    1975 non-null   object
 9   incident_address                1960 non-null   object
 10  street_name                     1960 non-null   object
 11  cross_street_1                  1610 non-null   object
 12  cross_street_2                  1613 non-null   